# Preprocessing 

In [23]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [172]:
df_train = pd.read_csv("../../data/train.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
df_valid = pd.read_csv("../../data/validation.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
df_test = pd.read_csv("../../data/test.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)

df = pd.concat([df_train, df_valid, df_test])

In [173]:
df.head()

,,,,,quadrant,lyrics
213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,"Gently, hold our heads Gently, hold our heads ..."
216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,We are the Sun We are the dead stars We are th...
254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,"You're out of touch, I'm out of time But I'm o..."
266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,Finally close the door You'd left open wide Lo...
283306,0.373325,-0.923151,Duke Ellington,Caravan,3,Night and stars above that shine so bright The...


In [174]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_preprocessing(text):
    text = text.lower() # lower text
    text = replace_contractions(text) # remove contactions
    text = "".join("".join(text).replace("\n", " ").replace("\r", " ")) # remove \n and \r
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace symbols with space
    text = BAD_SYMBOLS_RE.sub('', text) # replace bad characters with nothing
    text = text.strip()
    text = " ".join([word for word in text.split() if word not in STOPWORDS]) # remove stopwords
    
    return text

In [175]:
df["lyrics"] = df["lyrics"].apply(text_preprocessing)

In [176]:
df.head()

,,,,,quadrant,lyrics
213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,1,gently hold heads gently hold heads high aimle...
216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,1,sun dead stars black sky invading room candle ...
254391,0.815393,0.662457,Uniting Nations,Out Of Touch,0,touch time head around touch time head around ...
266313,1.257460,1.086515,The Walkmen,Brandy alexander,0,finally close door would left open wide locked...
283306,0.373325,-0.923151,Duke Ellington,Caravan,3,night stars shine bright mystery fading light ...


In [179]:
# get lens and remove anything with less than 10
df["len"] = df["lyrics"].apply(len)
df.describe()

,quadrant,len
count,17367.000000,17367.000000
mean,1.346980,1231.183336
std,1.117818,11910.935798
min,0.000000,0.000000
25%,0.000000,453.000000
50%,1.000000,621.000000
75%,2.000000,847.500000
max,3.000000,484931.000000


In [181]:
df = df.loc[df["len"] >= 10]
df.describe()

,quadrant,len
count,17366.000000,17366.000000
mean,1.347000,1231.254232
std,1.117847,11911.275087
min,0.000000,11.000000
25%,0.000000,453.250000
50%,1.000000,621.000000
75%,2.000000,847.750000
max,3.000000,484931.000000


In [183]:
df.drop(labels=["len"], axis=1, inplace=True)
df.head()

In [185]:
train_test = 0.1
train_valid = 0.8

train, test = train_test_split(df, test_size=train_test, random_state=0)
train, valid = train_test_split(train, train_size=train_valid, random_state=0)

In [186]:
train.shape, test.shape, valid.shape

((12503, 2), (1737, 2), (3126, 2))

In [187]:
train.to_csv("preprocessed_train.csv", index=False)
test.to_csv("preprocessed_test.csv", index=False)
valid.to_csv("preprocessed_valid.csv", index=False)

# Data Loading 

In [6]:
import torch
from torchtext.legacy.data import Field, TabularDataset, BucketIterator

In [7]:
BATCH_SIZE = 8

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
quadrants_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.long, is_target=True)
lyrics_field = Field(tokenize="spacy", lower=True, include_lengths=True, batch_first=True)
fields = [("quadrant", quadrants_field), ("lyrics", lyrics_field)]

trainset, validset, testset = TabularDataset.splits(path=".", train="preprocessed_train.csv",
                                                   validation="preprocessed_valid.csv",
                                                   test="preprocessed_test.csv", format="CSV",
                                                   fields=fields, skip_header=True)

/home/martinoywa/anaconda3/lib/python3.8/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [10]:
trainiter = BucketIterator(dataset=trainset, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.lyrics),
                            device=device, sort=True, sort_within_batch=True, train=True)

validiter = BucketIterator(dataset=validset, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.lyrics),
                            device=device, sort=True, sort_within_batch=True, train=True)

testiter = BucketIterator(dataset=testset, batch_size=BATCH_SIZE, sort_key=lambda x: len(x.lyrics),
                            device=device, sort=True, sort_within_batch=True, train=False)

In [11]:
# vocabulary
lyrics_field.build_vocab(trainset, min_freq=3)

# Model 

In [12]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import time

In [13]:
VOCAB_SIZE = len(lyrics_field.vocab)
EMBED_DIM = 3
HIDDEN_DIM = 64

In [14]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(LSTM, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=1, batch_first=True,
                           bidirectional=True)
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(2*hidden_dim, 4)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, lyrics, lyrics_len):
        embedded = self.embedding(lyrics)
#         print(lyrics[-1])
#         print()
#         print(lyrics_len)
        packed_in = pack_padded_sequence(embedded, lyrics_len.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.lstm(packed_in)
        output, _ = pad_packed_sequence(packed_out, batch_first=True)
        
        out_forward = output[range(len(output)), lyrics_len - 1, :HIDDEN_DIM]
        out_reverse = output[:, 0, HIDDEN_DIM:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.dropout(out_reduced)
        
        text_fea = self.fc(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        
#         print(text_fea.shape)
        
        return self.softmax(text_fea)

In [15]:
model = LSTM(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM).to(device)

In [16]:
def train_func(data):

    # Train the model
    train_loss = 0
    train_acc = 0
    for ((lyrics, lyrics_len), label) in data:
        optimizer.zero_grad()
        label, lyrics, lyrics_len = label.to(device), lyrics.to(device), lyrics_len.to(device)
        output = model(lyrics, lyrics_len)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == label).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(data), train_acc / len(data)

def test(data):
    loss = 0
    acc = 0
    for ((lyrics, lyrics_len), label) in data:
        label, lyrics, lyrics_len = label.to(device), lyrics.to(device), lyrics_len.to(device)
        with torch.no_grad():
            output = model(lyrics, lyrics_len)
            loss = criterion(output, label)
            loss += loss.item()
            acc += (output.argmax(1) == label).sum().item()

    return loss / len(data), acc / len(data)

In [17]:
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(trainiter)
    valid_loss, valid_acc = test(validiter)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

...

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([8, 4])
torch.Size([

KeyboardInterrupt: 